# **Data Augmentation**
Per aumentare gli elementi e la varietà del nostro dataset, utilizziamo alcune tecniche di data augmentation

In [ ]:
# Import generali
import os
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

### **1. Caricamento dei dati da Google drive**
Abbiamo due dataset, etichettati:


*   Il primo con 120 immagini in formato png
*   Il secondo con 249 immagini in formato jpg



In [ ]:
# Costanti che rappresentano le diverse cartelle e i formati delle nostre immagini
F = ['png', 'jpg']
D = ['Dysgraphic1', 'No_Dysgraphic1', 'Dysgraphic2', 'No_Dysgraphic2']

input_folder_path = f'/content/drive/MyDrive/datasetLabel/NoAugmentation/{D[3]}'
output_folder_path = f'/content/drive/MyDrive/datasetLabel/Augmentation/{D[3]}'

# Caricamento dei file all'interno della variabile images
file_paths = [os.path.join(input_folder_path, f) for f in os.listdir(input_folder_path) if f.endswith(f'.{F[1]}')]
images = [tf.image.decode_image(tf.io.read_file(fp)) for fp in file_paths]

augmented = []

# Stampe di test
print(str(len(file_paths)) + ' ' + file_paths[0])
print(len(images))
print(len(augmented))

### **2. Metodo di ritaglio delle immagini**
Con questo metodo creiamo tre nuove immagini, ritagliate in maniera casuale, a partire da ognuna delle immagini originali

In [ ]:
def crop_augmentation(original_images, augmented_images):
    for image in original_images:
        image = tf.cast(image, tf.float32) / 255.0

        # Estrazione delle dimensioni dell'immagine
        shape = tf.shape(image)
        height = tf.cast(shape[0], tf.float32)
        width = tf.cast(shape[1], tf.float32)

        # Calcolo dell'area di ritaglio, sulla base delle dimensioni estratte
        crop_height = tf.cast(height * 0.8, tf.int32)
        crop_width = tf.cast(width * 0.8, tf.int32)

        # Ritagli casuali
        for i in range(3):
            seed = (i, 0)
            random_crop = tf.image.stateless_random_crop(
                image, size=[crop_height, crop_width, 1], seed=seed)
            augmented_images.append(random_crop)

    # Stampe di test
    print(len(augmented_images))
    plt.imshow(augmented_images[0])
    plt.show()

    # Salvataggio delle immagini modificate
    for i, img in enumerate(augmented_images):
        image_data = tf.cast(img * 255, tf.uint8)
        numpy_image = image_data.numpy()
        rgb_image = cv2.cvtColor(numpy_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)
        pil_image.save(os.path.join(output_folder_path, f'crop_{i}.png'))

augmented = []
crop_augmentation(images, augmented)

### **3. Metodo di capovolgimento delle immagini**
Con questo metodo creiamo un immagine capovolta orizzontalmente per ognuna delle immagini originali

In [ ]:
def flip_augmentation(original_images, augmented_images):
    for image in original_images:

        # Capovolgimento dell'immagine
        flipped = tf.image.flip_left_right(image)
        augmented_images.append(flipped)

    # Stampe di test
    print(len(augmented_images))
    plt.imshow(augmented_images[0])
    plt.show()

    # Salvataggio delle immagini modificate
    for i, img in enumerate(augmented_images):
        image_data = tf.cast(img * 255, tf.uint8)
        numpy_image = image_data.numpy()
        rgb_image = cv2.cvtColor(numpy_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)
        pil_image.save(os.path.join(output_folder_path, f'flipped_{i}.png'))

augmented = []
flip_augmentation(images, augmented)

### **4. Metodo di rotazione delle immagini**
Con questo metodo creiamo un immagine ruotata, di un angolo casuale all'interno di un range specificato nella funzione, per ognuna delle immagini originali

In [ ]:
def rotation_augmentation(paths, augmented_images):
    # Range di angoli per la rotazione
    ang_range = 30

    for p in paths:
        image = cv2.imread(p)
        rows, cols, ch = image.shape

        # Rotazione dell'immagine
        ang_rot = np.random.uniform(ang_range)-ang_range/2
        Rot_M = cv2.getRotationMatrix2D((cols/2, rows/2),ang_rot,1)
        image = cv2.warpAffine(image, Rot_M, (cols,rows))
        augmented_images.append(image)

    # Stampe di test
    print(len(augmented_images))
    plt.imshow(augmented_images[0])
    plt.show()

    # Salvataggio delle immagini modificate
    for i, img in enumerate(augmented_images):
        rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)
        pil_image.save(os.path.join(output_folder_path, f'rotated_{i}.png'))

augmented = []
rotation_augmentation(file_paths, augmented)

### **5. Metodo di traslazione delle immagini**
Con questo metodo creiamo un immagine traslata, di un fattore casuale all'interno di un range specificato nella funzione, per ognuna delle immagini originali

In [ ]:
def translation_augmentation(paths, augmented_images):
    # Range di coordinate per la traslazione
    trans_range = 50

    for p in paths:
        image = cv2.imread(p)
        rows, cols, ch = image.shape

        # Traslazione
        tr_x = trans_range*np.random.uniform()-trans_range/2
        tr_y = trans_range*np.random.uniform()-trans_range/2
        Trans_M = np.float32([[1,0,tr_x], [0,1,tr_y]])
        image = cv2.warpAffine(image, Trans_M, (cols,rows))
        augmented_images.append(image)

    # Stampe di test
    print(len(augmented_images))
    plt.imshow(augmented_images[0])
    plt.show()

    # Salvataggio delle immagini modificate
    for i, img in enumerate(augmented_images):
        rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)
        pil_image.save(os.path.join(output_folder_path, f'translated_{i}.png'))

augmented = []
translation_augmentation(file_paths, augmented)

### **6. Metodo di aggiunta di rumore alle immagini**
Con questo metodo creiamo un immagine che aggiunge del rumore gaussiano a quella originale corrispondente

In [ ]:
def noise_augmentation(original_images, augmented_images):
    for image in original_images:

        # Aggiunta del rumore all'immagine
        noise = np.random.normal(0, 0.5, image.shape)
        noisy_image = image + noise
        noisy_image = np.clip(noisy_image, 0, 255)
        augmented_images.append(noisy_image)

    # Stampe di test
    print(len(augmented_images))
    plt.imshow(augmented_images[0])
    plt.show()

    # Salvataggio delle immagini modificate
    for i, img in enumerate(augmented_images):
        image_data = tf.cast(img * 255, tf.uint8)
        numpy_image = image_data.numpy()
        rgb_image = cv2.cvtColor(numpy_image, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(rgb_image)
        pil_image.save(os.path.join(output_folder_path, f'noisy_{i}.png'))

augmented = []
noise_augmentation(images, augmented)